# This notebook is holds helper functions for reading in, and processing, data files exported from mocap experiments from the Qualysis system.

In [2]:
import pandas as pd
import numpy as np

import scipy.signal as scisig
from scipy.stats import norm
from scipy.signal import butter, sosfreqz, sosfiltfilt, boxcar, savgol_filter, find_peaks
from scipy.signal.signaltools import hilbert

import os
import sys
import csv

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set the default figure sizes.
#
matplotlib.rc('figure', figsize=(15, 7))
sns.set(rc={'figure.figsize':(15,7)})

In [ ]:
def load_participant_data_from_path(filepath, debug=False):
    
    '''
        Loads data in from Qualysis exported (TSV) files, and returns a multi-index 
        dataframe.
        
        _____ INPUTS _____
        'filepath': Directory path to the data to load in (e.g. "./Data/Train")
        
        _____ OUTPUTS _____
        'data_df': A multi-index dataframe that has been populated by all data found
                   in the filepath.
    '''
    
    files = os.listdir(filepath)
    
    df_list = []
    for file in files:
        data_file = os.path.join(filepath, file)
        print("Reading in data from: ", data_file)

        # Grab attributes about the data that we will use to form the dataframe.
        #
        attrs = {}
        marker_names = []
        skip_cnt = 0
        with open(data_file) as fd:
            rd = csv.reader(fd, delimiter='\t')
            for i,row in enumerate(rd):
                # NOTE: 
                # - We always remove the first item in the 'row', as it's the name of the
                #   attribute we are interested in (e.g. 'MARKER_NAMES'), not the 
                #   value itself.
                #
                if "FREQUENCY" in row:
                    attrs['frequency'] = float(row[1:][0])
                    if debug:
                        print("Found sampling frequency\n", row[1:])
                if "DESCRIPTION" in row:
                    attrs['description'] = row[1].split(" - ")
                    if debug:
                        print("Found description\n", row[1:])
                if "TIME_STAMP" in row:
                    date = "".join(row[1].split(","))
                    attrs['date'] = date
                    if debug:
                        print("Found time stamp\n", row[1:])
                        print(date)
                if "MARKER_NAMES" in row:
                    if debug:
                        print("Found marker names\n", row[1:])
                    attrs['marker_names'] = row[1:]
                    skip_cnt = i+1
                    break

        # Create x,y,z names for each marker so we can populate the column names in the
        # dataframe correctly.
        # (e.g. 'hip_left' becomes 'hip_left_x', 'hip_left_y', 'hip_left_z')
        #
        marker_data_names = []
        for name in attrs['marker_names']:
            xyz = [name + "_x", name + "_y", name + "_z"]
            marker_data_names.extend(xyz)

        # Assign the new names to our attributes dictionary.
        #
        attrs['marker_data_names'] = marker_data_names
        if debug:
            print('\n', marker_data_names)

        # Read in the data and form the dataframe.
        #
        # df = pd.read_csv(data_file, sep='\t', names=attrs['marker_data_names'], skiprows=i+1, header=None)
        df = pd.read_csv(data_file, sep='\t', names=attrs['marker_data_names'], skiprows=skip_cnt+1, header=None)    

        # Create a multi-index so we can later group data easily.
        #
        cnt = len(df) # How many data points
        _participant = attrs['description'][0]
        _type = attrs['description'][1]
        _point_perf = attrs['description'][2]
        levels = [[_participant], [_type] , [_point_perf]]
        labels = [[0]*cnt for _ in levels]
        names = ['participant', 'type', 'point-perf.']
        index = pd.MultiIndex(levels=levels, labels=labels, names=names)
        df.index = index


        freq = str(1/(attrs['frequency'])) + "S"
        df['time'] = pd.DatetimeIndex(start=attrs['date'], periods=len(df), freq=freq)
        df.set_index('time', append=True, inplace=True)

        df_list.append(df)

    # Form the final dataframe that contains all data read in.
    #
    data_df = pd.concat(df_list)
    
    return data_df
    

In [ ]:
def form_spine_distance(df):
    ''''
        Calculate the Euclidean distance of each segment of the spine markers over all time of the 
        experiment. (i.e. markers s0->s1, s1->s2, s2->s3)
        
        _____ INPUTS _____
        'df': Dataframe containing columns that represent the individual spine markers in 3-d space.
        
        _____ OUTPUTS _____
        'dist_df': Dataframe containing the Euclidean distances that connects each marker to it's previous.
                   (i.e. the distance between s0->s1, s1->s2, etc.)
    '''
    
    dist_df = pd.DataFrame()
    # SegmentN is the distance from spine marker 0 (i.e. s_N) to the next 
    # spine marker (i.e. s_N+1).
    #
    dist_df['segment0'] = np.sqrt((df['spine_0_x'] - df['spine_1_x'])**2 + 
                            (df['spine_0_y'] - df['spine_1_y'])**2 +
                            (df['spine_0_z'] - df['spine_1_z'])**2)
    dist_df['segment1'] = np.sqrt((df['spine_1_x'] - df['spine_2_x'])**2 + 
                            (df['spine_1_y'] - df['spine_2_y'])**2 +
                            (df['spine_1_z'] - df['spine_2_z'])**2)
    dist_df['segment2'] = np.sqrt((df['spine_2_x'] - df['spine_3_x'])**2 + 
                            (df['spine_2_y'] - df['spine_3_y'])**2 +
                            (df['spine_2_z'] - df['spine_3_z'])**2)
    dist_df['dist_total'] = dist_df.sum(axis=1)
    
    return dist_df

In [ ]:
def scale_to_range(x, lower, upper):
    '''
        Scale data between a range defined by [lower, upper]
        
        _____ INPUTS _____
        'x': Array of data.
        'lower': Value of lower bound.
        'upper': Value of upper bound.
        
        _____ OUTPUTS _____
        Returns array scaled to sit between the range [lower, upper].
    '''
    return (x-min(x))*(upper-lower)/(max(x)-min(x)) + lower

In [ ]:
def standardize(x):
    return (x - x.mean())/x.std()

In [ ]:
def normalize(x):
    return (x - x.min())/(x.max()-x.min())

In [ ]:
# Create a set of filtering routines.
#

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    # Return second-order-sections for numerical stability.
    #
    sos = butter(N=order, Wn=[low, high], analog=False, btype='bandpass', output='sos')
    return sos

def butter_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order)
    filtered_signal = sosfiltfilt(sos, signal)
    return filtered_signal

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    cutoff_freq = cutoff / nyq
    # Return second-order-sections for numerical stability.
    #
    sos = butter(N=order, Wn=cutoff_freq, analog=False, btype='lowpass', output='sos')
    return sos

def butter_lowpass_filter(signal, cutoff, fs, order=5):
#     b, a = butter_lowpass(cutoff, fs, order)
    sos = butter_lowpass(cutoff, fs, order)
    filtered_signal = sosfiltfilt(sos, signal)
    return filtered_signal


In [ ]:
# Create a set of smoothing routines.
#

def boxcar_smooth(signal, window):
    # 'window': length of points
    
    return np.convolve(signal, boxcar(M=window))

def savgol_smooth(signal, window, order=3):
    # 'window': length of points
    #           NOTE: window must be odd.
    # 'order': polynomial order 
        
    if np.mod(window, 2) == 0:
        print("Window must be odd. Incrementing by 1")
        window += 1
    return savgol_filter(signal, window, order)